## Week 10 Assignment

#### Task1

Firstly, I create a csv file includinggiven information. Then I upload the file to Github. 

In [1]:
#set up the basic package
import numpy as np
import pandas as pd
#check the data set and loaded the required file from own repository.
url="https://raw.githubusercontent.com/steinszzh/DAV-5400/master/week10/Tidying%20and%20Transforming%20Data.csv"
flight = pd.read_csv(url)
#general check for the data
flight.head()

,Unnamed: 0,Unnamed: 1,Los Angeles,Phoenix,San Diego,San Francisco,Seattle
0,ALASKA,on time,497.0,221.0,212.0,503.0,1841.0
1,NaN,delayed,62.0,12.0,20.0,102.0,305.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AM WEST,on time,694.0,4840.0,383.0,320.0,201.0
4,NaN,delayed,117.0,415.0,65.0,129.0,61.0


#### Task2

Q:how would you define a “single observation” for the data shown in the graphic?; How many key values are associated with each data value?;
How many columns should your long format structure contain based on the information provided in the
graphic shown above?; What would the column headings for the long structure be?; etc.

Based on the requirement, I would like to get rid of these none values firstly. Then I will rename the category to get airline and Ontime or delay column.

In [2]:
flight.dropna(how='all',inplace=True)
#check the result
flight


,Unnamed: 0,Unnamed: 1,Los Angeles,Phoenix,San Diego,San Francisco,Seattle
0,ALASKA,on time,497.0,221.0,212.0,503.0,1841.0
1,NaN,delayed,62.0,12.0,20.0,102.0,305.0
3,AM WEST,on time,694.0,4840.0,383.0,320.0,201.0
4,NaN,delayed,117.0,415.0,65.0,129.0,61.0


In [3]:
# rename the columns 
flight.rename( columns={'Unnamed: 0':'Airline','Unnamed: 1':'Ontime or delay'}, inplace=True )
flight

,Airline,Ontime or delay,Los Angeles,Phoenix,San Diego,San Francisco,Seattle
0,ALASKA,on time,497.0,221.0,212.0,503.0,1841.0
1,NaN,delayed,62.0,12.0,20.0,102.0,305.0
3,AM WEST,on time,694.0,4840.0,383.0,320.0,201.0
4,NaN,delayed,117.0,415.0,65.0,129.0,61.0


After renaming the table, melt function is used to change it to long format.

In [4]:
# Melting from wide to long format, first 2 columns are the id
ids = ["Airline","Ontime or delay"]
# the rest are values
values = flight.columns[2:]
flight = pd.melt(frame=flight,id_vars=ids, value_vars=values,var_name='City', value_name='Value')
#check data
flight.head()



,Airline,Ontime or delay,City,Value
0,ALASKA,on time,Los Angeles,497.0
1,NaN,delayed,Los Angeles,62.0
2,AM WEST,on time,Los Angeles,694.0
3,NaN,delayed,Los Angeles,117.0
4,ALASKA,on time,Phoenix,221.0


In [5]:
#NaN value of delayed was filled by using on time value
flight['Airline'].fillna(method='ffill', inplace = True)
#check data
flight

,Airline,Ontime or delay,City,Value
0,ALASKA,on time,Los Angeles,497.0
1,ALASKA,delayed,Los Angeles,62.0
2,AM WEST,on time,Los Angeles,694.0
3,AM WEST,delayed,Los Angeles,117.0
4,ALASKA,on time,Phoenix,221.0
5,ALASKA,delayed,Phoenix,12.0
6,AM WEST,on time,Phoenix,4840.0
7,AM WEST,delayed,Phoenix,415.0
8,ALASKA,on time,San Diego,212.0
9,ALASKA,delayed,San Diego,20.0


As the result shows, there is ordered dataset now.

#### Task3

Using your reshaped/transformed data, perform analysis to compare the arrival delays for the two airlines. Some questions you might choose to answer: For each city, which airline had the best on time performance?; Which airline had the best overall on time performance?, etc.

Firstly, I need to extract data from previous data set. To get on time performance, I would like to get on time flight of each airline and the total values of flight amount to get percentage. 

In [6]:
# Extract total number of flights for ontime 
flight_ontime = flight[flight['Ontime or delay']=='on time'][['Airline','City','Value']]
flight_ontime

,Airline,City,Value
0,ALASKA,Los Angeles,497.0
2,AM WEST,Los Angeles,694.0
4,ALASKA,Phoenix,221.0
6,AM WEST,Phoenix,4840.0
8,ALASKA,San Diego,212.0
10,AM WEST,San Diego,383.0
12,ALASKA,San Francisco,503.0
14,AM WEST,San Francisco,320.0
16,ALASKA,Seattle,1841.0
18,AM WEST,Seattle,201.0


In [7]:
# create a frame to contain the total number of flights
flight_ontimetotal = flight.groupby(['Airline','City']).sum()['Value'].to_frame()
flight_ontimetotal

Value
Airline City                  
ALASKA  Los Angeles      559.0
        Phoenix          233.0
        San Diego        232.0
        San Francisco    605.0
        Seattle         2146.0
AM WEST Los Angeles      811.0
        Phoenix         5255.0
        San Diego        448.0
        San Francisco    449.0
        Seattle          262.0

Then use merge function to combine both two data sets.

In [8]:
#merge two d
merged_flight = pd.merge(flight_ontime, flight_ontimetotal, on=['Airline','City'],suffixes=(' ontime',' total'))
merged_flight

,Airline,City,Value ontime,Value total
0,ALASKA,Los Angeles,497.0,559.0
1,AM WEST,Los Angeles,694.0,811.0
2,ALASKA,Phoenix,221.0,233.0
3,AM WEST,Phoenix,4840.0,5255.0
4,ALASKA,San Diego,212.0,232.0
5,AM WEST,San Diego,383.0,448.0
6,ALASKA,San Francisco,503.0,605.0
7,AM WEST,San Francisco,320.0,449.0
8,ALASKA,Seattle,1841.0,2146.0
9,AM WEST,Seattle,201.0,262.0


After getting these two values, we can start to compute the percentage of ontime.

In [9]:
merged_flight['ontime_percentage'] = ((merged_flight['Value ontime'] * 100) / merged_flight['Value total'])
merged_flight.head()

,Airline,City,Value ontime,Value total,ontime_percentage
0,ALASKA,Los Angeles,497.0,559.0,88.908766
1,AM WEST,Los Angeles,694.0,811.0,85.573366
2,ALASKA,Phoenix,221.0,233.0,94.849785
3,AM WEST,Phoenix,4840.0,5255.0,92.102759
4,ALASKA,San Diego,212.0,232.0,91.379310


As the data showed, the percentage column is generated and the sequence just required the reorder process.

In [10]:
# sorted by the city 
sorted_flight = merged_flight.sort_values('City',ascending=True)
sorted_flight


,Airline,City,Value ontime,Value total,ontime_percentage
0,ALASKA,Los Angeles,497.0,559.0,88.908766
1,AM WEST,Los Angeles,694.0,811.0,85.573366
2,ALASKA,Phoenix,221.0,233.0,94.849785
3,AM WEST,Phoenix,4840.0,5255.0,92.102759
4,ALASKA,San Diego,212.0,232.0,91.379310
5,AM WEST,San Diego,383.0,448.0,85.491071
6,ALASKA,San Francisco,503.0,605.0,83.140496
7,AM WEST,San Francisco,320.0,449.0,71.269488
8,ALASKA,Seattle,1841.0,2146.0,85.787512
9,AM WEST,Seattle,201.0,262.0,76.717557


For each city, ALASKA airline has better performance than AM WEST. To compare the overall performance, we need to use get total on time flight and total flight for each airlines. Then compute to get each airline's overall performance.

In [11]:
#get total ontime number of flights for 'ALASKA'
Alaska_ontime = merged_flight[merged_flight['Airline']=='ALASKA'].sum()['Value ontime']

#get total number of flights for 'ALASKA'
Alaska_total = merged_flight[merged_flight['Airline']=='ALASKA'].sum()['Value total']


#get total ontime number of flights for 'Amwest'
Amwest_ontime = merged_flight[merged_flight['Airline']=='AM WEST'].sum()['Value ontime']

#get total number of flights for 'Amwest'
Amwest_total = merged_flight[merged_flight['Airline']=='AM WEST'].sum()['Value total']

#get overall percentage of ontime flights for 'ALASKA'
Alaska_perf=Alaska_ontime/Alaska_total

Alaska_perf

0.8672847682119206

In [12]:
#get overall percentage of ontime flights for 'AM WEST'

Amwest_perf=Amwest_ontime/Amwest_total
Amwest_perf

0.8910726643598615

Interestingly, AM WEST has a better overall ontime performance even though AM WEST lose on every individual city. Also, Am West is 2% more higher than ALASKA overall performance.

#### Task4

Finally, given your “tidy” long format structure, consider what, if any, changes you would make to the visual presentation of the data if you were then asked to transform your “long” data back into a “wide” format: would you mimic the structure of the graphic shown above? If not, how might you transform your “long” data to “wide” format to make its “wide” presentation easier to understand and work with? Provide an example of your recommendation.

Only when I want to analyze the performance of a specific city, I may change it back. 
I would like to use pivot function to transform 'long' to 'wide' format. Then use unstack function to make it look nice.

In [13]:
# create a pivot table by mimic the origianl format 
flight_pivot= pd.pivot_table(flight,values='Value',index=['Airline','Ontime or delay'],columns=['City'])
flight_pivot


City                     Los Angeles   Phoenix  San Diego  San Francisco   \
Airline Ontime or delay                                                     
ALASKA  delayed                  62.0     12.0       20.0           102.0   
        on time                 497.0    221.0      212.0           503.0   
AM WEST delayed                 117.0    415.0       65.0           129.0   
        on time                 694.0   4840.0      383.0           320.0   

City                     Seattle  
Airline Ontime or delay           
ALASKA  delayed            305.0  
        on time           1841.0  
AM WEST delayed             61.0  
        on time            201.0

In [14]:
#use unstack function to unstack wide

flight_wide = pd.DataFrame(flight_pivot.unstack())
#get wide
flight_wide

City            Los Angeles          Phoenix         San Diego          \
Ontime or delay      delayed on time delayed on time   delayed on time   
Airline                                                                  
ALASKA                  62.0   497.0    12.0   221.0      20.0   212.0   
AM WEST                117.0   694.0   415.0  4840.0      65.0   383.0   

City            San Francisco          Seattle          
Ontime or delay        delayed on time delayed on time  
Airline                                                 
ALASKA                   102.0   503.0   305.0  1841.0  
AM WEST                  129.0   320.0    61.0   201.0

As the result show, now it is the wide format.